<center> <h1>Illustrating alpha and information ratio in Homework 2</h1> </center>

<center>William Mann</center>

<center>
<img src="https://wgjm1986.github.io/FIN323/Emory_small.png" height="200px" width="300px" />
</center>                                                                                            

In [1]:
%%capture

import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'

import os
import wrds
import fredapi
import math
import numpy as np

import statsmodels.api as sm

## Application: The analysis of Homework 2

We will reproduce the results of that assignment, and then see how the formulas from this week are another path to the same answers.

In [2]:
conn = wrds.Connection(wrds_username='wmann')

Loading library list...
Done


In [3]:
# Risk-free rate from the Fama and French data
FF3F = conn.get_table(library='ff_all',table='factors_monthly')
FF3F['month'] = pd.to_datetime(FF3F['date']).dt.to_period('M')
FF3F = FF3F.drop('date',axis=1).drop('dateff',axis=1).set_index('month')
riskfree = FF3F.rf.astype('float64')
riskfree = riskfree['2008':'2024']

In [4]:
Gold = conn.raw_sql("""select comcode, of, date_ from trdstrm.wrds_cmdy_data 
    where comcode = '1865' and of is not null and of <> 5""")
Gold['date_'] = pd.to_datetime(Gold['date_'])
Gold = Gold.sort_values(by='date_')
Gold = Gold.reset_index().resample('ME',on='date_').agg(['last'])
Gold['month'] = Gold.index.to_period('M')
Gold['ret'] = (Gold['of']/Gold['of'].shift())-1
Gold = Gold.set_index('month')
Gold = Gold['ret']
Gold = round(Gold,6)
Gold = Gold['2008':'2024']

In [5]:
# VFINX returns from WRDS
VFINX_crsp_fundno = conn.raw_sql("select crsp_fundno, ticker from crsp_q_mutualfunds.fund_hdr where ticker = 'VFINX'").reset_index().crsp_fundno[0]
VFINX = conn.raw_sql("select caldt, mret from crsp_q_mutualfunds.monthly_returns where crsp_fundno = " + str(VFINX_crsp_fundno))
# Convert string date to datetime
VFINX = VFINX.set_index( pd.to_datetime(VFINX.set_index('caldt').index) ).drop('caldt',axis=1)
# Aggregate to monthly level
VFINX['month'] = VFINX.index.to_period('M')
VFINX = (VFINX.set_index('month') + 1).groupby('month').prod() - 1
VFINX.columns = ['VFINX']
VFINX = VFINX['2008':'2024']
VFINX = VFINX['VFINX']

In [6]:
conn.close()

In [7]:
# Optional: Write data to file for in-class Excel example

# try: os.remove("Homework 2 example.xlsx")
# except OSError: pass

# # Combine all datasets together
# HWdata = pd.concat([VFINX.to_frame(name='VFINX'), Gold.to_frame(name='Gold'), riskfree.to_frame(name='rf')],axis=1)

# # set up custom ExcelWriter engine to get the date format correct in the output file
# writer = pd.ExcelWriter('Homework 2 example.xlsx', engine='xlsxwriter', datetime_format= "yyyy-mm")

# HWdata.to_excel(writer,sheet_name='Data')

# writer.close()

I have already downloaded the data on VFINX, gold, and risk-free returns.

In the homework we found the portfolio with the highest Sharpe ratio. 
Let's quickly calculate that answer again, using the matrix formulas form last week:

In [8]:
Gold_excess = Gold - riskfree
Gold_excess.name = 'Gold excess return'
VFINX_excess = VFINX - riskfree
VFINX_excess.name = 'VFINX excess return'

In [9]:
Sigma = np.cov([ Gold_excess , VFINX_excess ])
mu = [ Gold_excess.mean() , VFINX_excess.mean() ]
Optimal_weights_unscaled = np.linalg.inv(Sigma) @ mu
Optimal_weights = (
    Optimal_weights_unscaled / Optimal_weights_unscaled.sum()
    )
Optimal_Gold_weight, Optimal_VFINX_weight = Optimal_weights

print("The portfolio with the highest Sharpe ratio is:")
print(f" {100*Optimal_Gold_weight :.3f}% Gold,")
print(f" {100*Optimal_VFINX_weight :.3f}% VFINX.")

Optimal_mu = Optimal_weights @ mu
Optimal_sigma = np.sqrt( Optimal_weights @ (Sigma @ Optimal_weights) )
Optimal_SR = Optimal_mu / Optimal_sigma

print(f"It has a Sharpe ratio of {Optimal_SR :.5f}.")

The portfolio with the highest Sharpe ratio is:
 36.687% Gold,
 63.313% VFINX.
It has a Sharpe ratio of 0.21513.


### Using the Sharpe ratio formula from above

Now let's look at the first formula from this week. It say that we want to increase allocation to any investment $i$ as long as $SR_i > \rho_{ip} \times SR_p$, where $p$ is the portfolio we currently hold.

Let's apply this formula to the homework analysis, with gold playing the role of investment $i$ and VFINX the portfolio $p$.

First we calculate the Sharpe ratio of gold:

In [10]:
Gold_SR = Gold_excess.mean() / Gold_excess.std()
print(f"Sharpe ratio of gold: {Gold_SR :.5f}")

Sharpe ratio of gold: 0.12038


Now imagine we start off holding just VFINX, and are considering whether to allocate some to gold.
Calculate the Sharpe ratio of VFINX by itself, and the correlation of gold with VFINX.

In [11]:
VFINX_SR = VFINX_excess.mean() / VFINX_excess.std()
Corr_Gold_VFINX = np.corrcoef(
    [ VFINX_excess , Gold_excess ]
    )[0,1]

print(f"Sharpe ratio of VFINX: {VFINX_SR :.5f}")
print(f"Correlation of gold with VFINX: {Corr_Gold_VFINX :.5f}")
print(f"Multiply these two together: {VFINX_SR*Corr_Gold_VFINX :.5f}")

Sharpe ratio of VFINX: 0.18518
Correlation of gold with VFINX: 0.05986
Multiply these two together: 0.01109


Because the Sharpe ratio of gold was greater than this, we conclude that we can achieve a higher portfolio Sharpe ratio if we allocate *some* amount to gold.

Now suppose we try an allocation of 20% to gold. We can simulate the returns of this allocation, and then redo the 
calculations just above with this new allocation playing the role of $p$, our current portfolio.

In [12]:
Gold20_excess = 0.2*Gold_excess + 0.8*VFINX_excess
Gold20_excess.name = '20% Gold / 80% VFINX excess return'
Gold20_SR = Gold20_excess.mean() / Gold20_excess.std()
Corr_Gold_Gold20 = np.corrcoef(
    [ Gold_excess , Gold20_excess ]
    )[0,1]

print(f"Sharpe ratio of an allocation 20% gold, 80% VFINX: {Gold20_SR :.5f}")
print(f"Correlation of gold with this allocation: {Corr_Gold_Gold20 :.5f}")
print(f"Multiply these two together: {Gold20_SR*Corr_Gold_Gold20 :.5f}")

Sharpe ratio of an allocation 20% gold, 80% VFINX: 0.20679
Correlation of gold with this allocation: 0.30943
Multiply these two together: 0.06399


The result is still less than the Sharpe ratio of gold by itself that we calculated earlier. So we conclude that we should allocate more than 20% to gold. 

We can repeat this process by trial and error.
It will eventually settle down to the answer we found in Homework 2.
Let's verify that once we reach that answer, the two sides of the equation are exactly equal.

In [13]:
Optimal_excess = Optimal_weights @ [ Gold_excess , VFINX_excess ]
Optimal_excess = pd.Series(Optimal_excess, name = 'Optimal portfolio excess return', index = Gold_excess.index)

Optimal_SR = Optimal_excess.mean() / Optimal_excess.std()
Corr_Optimal_Gold = np.corrcoef(
    [ Optimal_excess , Gold_excess ]
    )[0,1]

print(f"Sharpe ratio of gold: {Gold_SR :.5f}")
print(f"Sharpe ratio of optimal portfolio from HW2: {Optimal_SR :.5f}")
print(f"Correlation of optimal portfolio with gold: {Corr_Optimal_Gold :.5f}")
print(f"Multiply the last two numbers: {Optimal_SR*Corr_Optimal_Gold :.5f}" )

Sharpe ratio of gold: 0.12038
Sharpe ratio of optimal portfolio from HW2: 0.21513
Correlation of optimal portfolio with gold: 0.55958
Multiply the last two numbers: 0.12038


So this allocation to gold achieves the highest possible Sharpe ratio in our historical data.

## Formula with alpha

The next formula from this week said that we want to add more allocation to an investment $i$ as long as it has positive $\alpha$ with respect to our current portfolio.

$\alpha$ measures how much an investment outperforms its hurdle rate:
$$
\alpha_{ip} = \mathbb{E}[r_i-r_f] - \beta_{ip}\times\mathbb{E}[r_p-r_f]
$$

We can estimate it in either of two ways:

1. Substitute the average excess returns on $i$ and $p$ into the expected values above.

2. Get the intercept from a regression of excess returns of $i$ on excess returns of $p$,

$$
(r_{it}-r_{ft}) = \alpha_{ip} + \beta_{ip}\times(r_{pt} - r_{ft}) + \varepsilon_{ipt}
$$

Let's see how these formulas show up in the analysis from Homework 2.

First we check that gold has a positive $\alpha$ with respect to VFINX. In the regression table below, the number labeled "Intercept" is our estimate of $\alpha$. The number labeled VFINX is our estimate of $\beta$.

In [14]:
# first define a couple of helper functions for regressions (the statsmodels syntax is a bit arcane)
# run a linear regression:
def regress(y,x): return sm.OLS( y , sm.add_constant(x).rename(columns={'const':'Intercept'}) ).fit()
# print a table of output:
def regression_table(y,x): return regress(y,x).summary().tables[1]

In [15]:
regression_table( Gold_excess , VFINX_excess )

<class 'statsmodels.iolib.table.SimpleTable'>

We can interpret $\alpha$ as a monthly return. So, we would say that, in our historical data, gold delivers 0.53% per month more than its appropriate hurdle rate based on its $\beta_{ip}$ with respect to our current portolio of holding only VFINX. 

The value of $\beta$ is very low and effectively zero. This tells us that gold has almost no correlation with the performance of the overall stock market (represented by VFINX), even though gold is very volatile on its own.

Since we found a positive $\alpha$ for gold in the regression, this tells us that we can increase our Sharpe ratio by adding some allocation to gold to our portfolio, starting from zero allocation, and so we should.

Let's quickly check that we can match the alpha and beta value from the regression above by calculating directly with the formulas from slide 4:

In [16]:
Corr_Gold_VFINX = np.corrcoef(
    [ Gold_excess , VFINX_excess ]
    )[0,1]
Vol_Gold = Gold_excess.std()
Vol_VFINX = VFINX_excess.std()
Beta_Gold_VFINX = Corr_Gold_VFINX * Vol_Gold / Vol_VFINX
Alpha_Gold_VFINX = (
    Gold_excess.mean() - Beta_Gold_VFINX * VFINX_excess.mean()
    )
print("Matching the above regression by direct calculations:")
print(f"  Beta: {Beta_Gold_VFINX :.4f}")
print(f"  Alpha: {Alpha_Gold_VFINX :.4f}")

Matching the above regression by direct calculations:
  Beta: 0.0634
  Alpha: 0.0053


Now we again try our allocation of 20% to gold, 80% to VFINX, and evaluate the $\alpha$ of gold with respect to this allocation:

In [17]:
regression_table(Gold_excess,Gold20_excess)

<class 'statsmodels.iolib.table.SimpleTable'>

There is still some evidence of positive $\alpha$, suggesting that we could do even better by increasing our allocation to gold some more.

Finally, we can check that gold has exactly zero alpha with respect to our optimal solution from Homework 2, so we could find that same answer by tinkering with the portfolio until alpha settles down to zero:

In [18]:
regression_table( Gold_excess, Optimal_excess )

<class 'statsmodels.iolib.table.SimpleTable'>

So far we have matched the answers from Homework 2 using two different formulas, one based on Sharpe ratio and one based on $\alpha$.

These two approaches felt very similar, because they are really the same thing, just stating the math in a different way.

The second approach explains why $\alpha$ is used as a general measure of investment performance.

### Analysis based on information ratios

The information ratio is $\alpha$ scaled by a measure of risk. Where $\alpha$ just says whether it's *possible* to improve your Sharpe ratio using a new investment, the information ratio tells us *how much* your performance can improve using that investment, and how much you want to allocate to it.

The information ratio can help investors judge whether it's actually worthwhile to pursue a new investment. By the same token, active managers should focus on achieving a high information ratio with respect to their benchmark, because this is how they will attract more funds.

We start out by calculating the information ratio of gold with respect to VFINX:

In [19]:
regression_results = regress(Gold_excess,VFINX_excess)
alpha = regression_results.params.iloc[0]
resid_std = regression_results.resid.std()
Gold_IR = alpha / resid_std
print(f"Information ratio of gold with respect to VFINX: {Gold_IR :.5f}")

Information ratio of gold with respect to VFINX: 0.10949


There are many ways to use this number. One of the most important is a formula we saw above: For an investor who allocates only between the new investment $i$ (in this case gold), and another portfolio $p$ (in this case VFINX), the maximum Sharpe ratio they can achieve is given by the following formula (slide 6 and textbook formula 8.26),
$$
SR_{max} = \sqrt{ SR_p^2 + IR_{ip}^2 }
$$
Using the numbers we have calculated, we calculate a maximum Sharpe ratio that exactly matches the Sharpe ratio of our solution from Homework 2:

In [21]:
Max_SR = np.sqrt(VFINX_SR**2 + Gold_IR**2)
print(f"  Max SR: { Max_SR :.5f}")

  Max SR: 0.21513


The point is that we can decide how attractive this new investment is, without doing an actual optimization.

If a manager has a good guess what portfolio their clients hold on average (it is likely close to the market portfolio!), then they should maximize their information ratio to attract the most investment into their fund.

One last useful connection with regression theory: The information ratio is really just the t-statistic of the regression intercept multiplied by $\sqrt{N}$, where $N$ is the number of observations of data in the regression. (But note, to make this formula line up perfectly requires some further small adjustments that are beyond the scope of the class right now.)